In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

DIR = "/content/drive/MyDrive/Competitions/Signate/MUFG2023"
INPUT_DIR = os.path.join(DIR,"input")
OUTPUT_DIR = os.path.join(DIR,"output")
OOF_DIR = os.path.join(OUTPUT_DIR, 'oof')
SUB_NUM_DIR = os.path.join(OUTPUT_DIR, 'sub_num')

OUTPUT_EXP26_DIR = DIR + '/output/EXP026/'
OUTPUT_EXP53_DIR = DIR + '/output/EXP053/'
OUTPUT_EXP54_DIR = DIR + '/output/EXP054/'
OUTPUT_EXP70_DIR = DIR + '/output/EXP070/'
OUTPUT_EXP71_DIR = DIR + '/output/EXP071/'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.simplefilter('ignore')

In [4]:
#!pip uninstall lightgbm --yes

#!pip install lightgbm \
#    --no-binary lightgbm \
#    --no-cache lightgbm \
#    --config-settings=cmake.define.USE_CUDA=ON

In [5]:
!pip install feature_engine
!pip install optuna

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GroupKFold, StratifiedGroupKFold, KFold
from sklearn.metrics import mean_squared_error,  mean_absolute_percentage_error
from sklearn.metrics import f1_score, accuracy_score

from sklearn import preprocessing
from feature_engine.encoding import WoEEncoder

import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00


In [6]:
train = pd.read_csv(os.path.join(INPUT_DIR,"train.csv"))
test = pd.read_csv(os.path.join(INPUT_DIR,"test.csv"))
card = pd.read_csv(os.path.join(INPUT_DIR, "card.csv"))
user = pd.read_csv(os.path.join(INPUT_DIR, "user.csv"))
sub = pd.read_csv(os.path.join(INPUT_DIR, "sample_submit.csv"), header=None)

print(train.shape)
display(train.head(3))

print(test.shape)
display(test.head(3))

print(card.shape)
display(card.head(3))

print(user.shape)
display(user.head(3))

print(sub.shape)
display(sub.head(3))

(471283, 12)


,index,user_id,card_id,amount,errors?,is_fraud?,merchant_id,merchant_city,merchant_state,zip,mcc,use_chip
0,0,1721,0,$2.623,OK,0,209237,Joliet,IL,60436.0,5541,Swipe Transaction
1,1,1629,3,$6.4,OK,0,2568,Edgerton,WI,53534.0,5814,Swipe Transaction
2,2,655,3,$123.5,OK,0,345310,Ridgefield,WA,98642.0,7538,Swipe Transaction


(457958, 11)


,index,user_id,card_id,amount,errors?,merchant_id,merchant_city,merchant_state,zip,mcc,use_chip
0,471283,541,3,$113.278,OK,324189,Orlando,FL,32821.0,4814,Swipe Transaction
1,471284,655,1,$293.944,OK,81219,Ridgefield,WA,98642.0,7538,Chip Transaction
2,471285,492,0,$47.4,OK,274755,Arlington Heights,IL,60004.0,5719,Swipe Transaction


(416, 10)


,user_id,card_id,card_brand,card_type,expires,has_chip,cards_issued,credit_limit,acct_open_date,year_pin_last_changed
0,39,0,Visa,Debit,09/2021,YES,1,$17117,05/2007,2010
1,39,1,Amex,Credit,11/2024,YES,2,$5400,10/2015,2015
2,41,0,Discover,Credit,03/2022,YES,2,$14800,12/2010,2011


(97, 17)


,user_id,current_age,retirement_age,birth_year,birth_month,gender,address,city,state,zipcode,latitude,longitude,per_capita_income_zipcode,yearly_income_person,total_debt,fico_score,num_credit_cards
0,39,57,64,1962,12,Female,442 Burns Boulevard,Mansfield,MA,2048,42.02,-71.21,$37407,$76274,$102611,698,2
1,41,39,66,1980,10,Female,3863 River Avenue,Lincoln,CA,95648,38.93,-121.25,$21829,$44506,$57994,849,3
2,47,40,67,1979,5,Female,8799 Elm Avenue,Mckinney,TX,75069,33.20,-96.65,$24684,$50329,$76759,625,4


(457958, 2)


,0,1
0,471283,0
1,471284,1
2,471285,0


In [7]:
v3_pred_df = pd.read_pickle(OUTPUT_EXP26_DIR+'oof_df.pkl')
v3_pred_df.rename(columns={"pred":"v3_pred"},inplace=True)
# v3_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_microsoft-deberta-v3-base_seed42_ver26.csv"), header=None)
# v3_test.columns = ["index", "pred", "is_fraud?",	"pred_v3_fold0",	"pred_v3_fold1",	"pred_v3_fold2",	"pred_v3_fold3",	"pred_v3_fold4"]

v1_pred_df = pd.read_pickle(OUTPUT_EXP53_DIR+'oof_df.pkl')
v1_pred_df.rename(columns={"pred":"v1_pred"},inplace=True)
# v1_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_microsoft-deberta-base_seed42_ver53.csv"), header=None)
# v1_test.columns = ["index", "pred", "is_fraud?",	"pred_v1_fold0",	"pred_v1_fold1",	"pred_v1_fold2",	"pred_v1_fold3",	"pred_v1_fold4"]

roberta_pred_df = pd.read_pickle(OUTPUT_EXP54_DIR+'oof_df.pkl')
roberta_pred_df.rename(columns={"pred":"roberta_pred"},inplace=True)
# roberta_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_roberta-base_seed42_ver54.csv"), header=None)
# roberta_test.columns = ["index", "pred", "is_fraud?",	"pred_roberta_fold0",	"pred_roberta_fold1",	"pred_roberta_fold2",	"pred_roberta_fold3",	"pred_roberta_fold4"]

funnel_pred_df = pd.read_pickle(OUTPUT_EXP70_DIR+'oof_df.pkl')
funnel_pred_df.rename(columns={"pred":"funnel_pred"},inplace=True)
# funnel_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_funnel-transformer-medium_seed42_ver70.csv"), header=None)
# funnel_test.columns = ["index", "pred", "is_fraud?",	"pred_funnel_fold0",	"pred_funnel_fold1",	"pred_funnel_fold2",	"pred_funnel_fold3",	"pred_funnel_fold4"]

electra_pred_df = pd.read_pickle(OUTPUT_EXP71_DIR+'oof_df.pkl')
electra_pred_df.rename(columns={"pred":"electra_pred"},inplace=True)
# electra_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_google-electra-base-discriminator_seed42_ver71.csv"), header=None)
# electra_test.columns = ["index", "pred", "is_fraud?",	"pred_electra_fold0",	"pred_electra_fold1",	"pred_electra_fold2",	"pred_electra_fold3",	"pred_electra_fold4"]

In [8]:
oof56_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_logistic_ver56.csv'))
oof56_df.rename(columns={"pred":"oof56_logistic_pred"},inplace=True)
# oof56_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_logistic_ver56.csv"))

oof65_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_ridge_ver65.csv'))
oof65_df.rename(columns={"pred":"oof65_ridge_pred"},inplace=True)
# oof65_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_ridge_ver65.csv"))

oof57_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_knn_ver57.csv'))
oof57_df.rename(columns={"pred":"oof57_knn3_pred"},inplace=True)
# oof57_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_knn_ver57.csv"))

oof58_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_knn_ver58.csv'))
oof58_df.rename(columns={"pred":"oof58_knn5_pred"},inplace=True)
# oof58_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_knn_ver58.csv"))

oof59_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_knn_ver59.csv'))
oof59_df.rename(columns={"pred":"oof59_knn8_pred"},inplace=True)
# oof59_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_knn_ver59.csv"))

oof60_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_knn_ver60.csv'))
oof60_df.rename(columns={"pred":"oof60_knn12_pred"},inplace=True)
# oof60_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_knn_ver60.csv"))

oof69_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_rf_ver69.csv'))
oof69_df.rename(columns={"pred":"oof69_rf_pred"},inplace=True)
# oof69_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_rf_ver69.csv"))

oof74_df = pd.read_csv(os.path.join(OOF_DIR, f'oof_et_ver74.csv'))
oof74_df.rename(columns={"pred":"oof74_et_pred"},inplace=True)
# oof74_test = pd.read_csv(os.path.join(SUB_NUM_DIR, f"submit_num_et_ver74.csv"))

In [9]:
v3_pred_df["oof56_logistic_pred"] = oof56_df["oof56_logistic_pred"]
v3_pred_df["oof65_ridge_pred"] = oof65_df["oof65_ridge_pred"]
v3_pred_df["oof57_knn3_pred"] = oof57_df["oof57_knn3_pred"]
v3_pred_df["oof58_knn5_pred"] = oof58_df["oof58_knn5_pred"]
v3_pred_df["oof59_knn8_pred"] = oof59_df["oof59_knn8_pred"]
v3_pred_df["oof60_knn12_pred"] = oof60_df["oof60_knn12_pred"]
v3_pred_df["oof69_rf_pred"] = oof69_df["oof69_rf_pred"]
v3_pred_df["oof74_et_pred"] = oof74_df["oof74_et_pred"]

In [10]:
train = train.merge(v3_pred_df[["index", "v3_pred","oof56_logistic_pred", "oof65_ridge_pred", "oof57_knn3_pred", "oof58_knn5_pred", "oof59_knn8_pred", "oof60_knn12_pred", "oof69_rf_pred", "oof74_et_pred", "kfold"]], how="left", on="index")\
.merge(v1_pred_df[["index", "v1_pred", "kfold"]], how="left", on=["index","kfold"]).merge(roberta_pred_df[["index", "roberta_pred", "kfold"]], how="left", on=["index","kfold"]).merge(funnel_pred_df[["index", "funnel_pred", "kfold"]], how="left", on=["index","kfold"])\
.merge(electra_pred_df[["index", "electra_pred", "kfold"]], how="left", on=["index","kfold"])

In [11]:
train.head(3)

,index,user_id,card_id,amount,errors?,is_fraud?,merchant_id,merchant_city,merchant_state,zip,...,oof58_knn5_pred,oof59_knn8_pred,oof60_knn12_pred,oof69_rf_pred,oof74_et_pred,kfold,v1_pred,roberta_pred,funnel_pred,electra_pred
0,0,1721,0,$2.623,OK,0,209237,Joliet,IL,60436.0,...,0.0,0.0,0.0,0.027680,0.034623,3,0.007156,0.009872,0.006189,0.012007
1,1,1629,3,$6.4,OK,0,2568,Edgerton,WI,53534.0,...,0.0,0.0,0.0,0.020071,0.026112,0,0.002065,0.001569,0.002745,0.002182
2,2,655,3,$123.5,OK,0,345310,Ridgefield,WA,98642.0,...,0.0,0.0,0.0,0.028444,0.033529,2,0.029314,0.060902,0.035393,0.038071


In [12]:
month_dict = {
   "01": 1,
   "02": 2,
   "03": 3,
   "04": 4,
   "05": 5,
   "06": 6,
   "07": 7,
   "08": 8,
   "09": 9,
   "10": 10,
   "11": 11,
   "12": 12
}

def get_expires_values(df):
  _df = df["expires"].str.split('/').apply(pd.Series)
  _df.columns = ["month","years"]
  df["expires_month"] = _df["month"].astype(str)
  df["expires_years"] = _df["years"].astype(int)
  return df

def get_acct_open_date_values(df):
  _df = df["acct_open_date"].str.split('/').apply(pd.Series)
  _df.columns = ["month","years"]
  df["acct_open_date_month"] = _df["month"].astype(str)
  df["acct_open_date_years"] = _df["years"].astype(int)
  return df


card = get_expires_values(card)
card = get_acct_open_date_values(card)
card["expires_month"] = card["expires_month"].map(month_dict)
card["acct_open_date_month"] = card["acct_open_date_month"].map(month_dict)

card["dt_acct_open_date"] = pd.to_datetime(card["acct_open_date"])
card["dt_expires"] = pd.to_datetime(card["expires"])

tmps = []
card["index"] = 1
for id in list(card.user_id.unique()):
  tmp = card.loc[card["user_id"]==id].sort_values("dt_acct_open_date")
  tmp["have_card"] = tmp["index"].cumsum()  # x枚目のカード
  tmps.append(tmp[["user_id", "card_id", "have_card"]])
card_h = pd.concat(tmps)
card = card.merge(card_h, how="left", on=["user_id", "card_id"])

In [13]:
card["credit_limit"] = card["credit_limit"].astype(str).str.extract("(\d+)").astype("float32")
user["per_capita_income_zipcode"] = user["per_capita_income_zipcode"].astype(str).str.extract("(\d+)").astype("float32")
user["yearly_income_person"] = user["yearly_income_person"].astype(str).str.extract("(\d+)").astype("float32")
user["total_debt"] = user["total_debt"].astype(str).str.extract("(\d+)").astype("float32")

In [14]:
use_limit = card.groupby("user_id")["credit_limit"].sum().to_dict()
card["credit_limit_sum"] = card["user_id"].map(use_limit)

In [15]:
train["amount"] = train["amount"].astype(str).str.extract("(\d+)").astype("float32")
test["amount"] = test["amount"].astype(str).str.extract("(\d+)").astype("float32")

In [16]:
concat_data = pd.concat([train,test],axis=0)
concat_data = concat_data.merge(card, how="left", on=["user_id", "card_id"]).merge(user, how="left", on="user_id")

concat_data = concat_data.fillna("NAN")

In [17]:
# 買った町と住んでいる町が一致していれば1
concat_data["merchant_address_city"] = 0
concat_data.loc[concat_data["merchant_city"]==concat_data["city"], "merchant_address_city"] = 1

# オンライン１　現地0
concat_data["is_ONLINE"] = 0
concat_data.loc[concat_data["merchant_city"]=="ONLINE", "is_ONLINE"] = 1

# 家の住所と購買距離
lat_state_mean = concat_data.groupby("state")["latitude"].mean().to_dict()
lon_state_mean = concat_data.groupby("state")["longitude"].mean().to_dict()
lat_mean = concat_data["latitude"].mean()
lon_mean = concat_data["longitude"].mean()
concat_data["merchant_lat"] = concat_data["merchant_state"].map(lat_state_mean)
concat_data["merchant_lon"] = concat_data["merchant_state"].map(lon_state_mean)
concat_data["merchant_lat"] = concat_data["merchant_lat"].fillna(lat_mean)
concat_data["merchant_lon"] = concat_data["merchant_lon"].fillna(lon_mean)
concat_data["merchant_address_distance"] = np.sqrt((concat_data["latitude"]-concat_data["merchant_lat"])**2 + (concat_data["longitude"]-concat_data["merchant_lon"])**2)

In [18]:
city_c = concat_data["merchant_city"].value_counts(normalize=True).to_dict()
state_c = concat_data["merchant_state"].value_counts(normalize=True).to_dict()

concat_data["city_c"] = concat_data["merchant_city"].map(lambda x: city_c[x]).values
concat_data["state_c"] = concat_data["merchant_state"].map(lambda x: state_c[x]).values

concat_data.loc[concat_data["city_c"]<0.0005, "merchant_city"] = "Rare_city"
concat_data.loc[concat_data["state_c"]<0.0005, "merchant_state"] = "Rare_state"

In [19]:
for col in ["errors?", "merchant_city", "merchant_state", "use_chip", "card_brand", "card_type", "has_chip", "gender", "city", "state"]:
  print(col)
  le = preprocessing.LabelEncoder()
  le.fit(concat_data[col])
  concat_data[col] = le.transform(concat_data[col])

errors?
merchant_city
merchant_state
use_chip
card_brand
card_type
has_chip
gender
city
state


In [20]:
income_mean = concat_data["yearly_income_person"].mean()
concat_data["income_all_rate"] = concat_data["yearly_income_person"] / income_mean  # 平均年収の稼いでいる割合
concat_data["income_rate"] = concat_data["yearly_income_person"] / concat_data["per_capita_income_zipcode"]  # 市町村の平均年収の何倍その人が稼いでいるか
concat_data["debt_rate"] = concat_data["total_debt"] / concat_data["yearly_income_person"]  # ローン総額の年収に対する割合
concat_data["limit_rate"] = concat_data["credit_limit_sum"] / concat_data["yearly_income_person"]  # 利用上限総額に対する年収の割合
concat_data["amount_rate"] = concat_data["amount"] / concat_data["credit_limit"]   # 買い物した額の上限額の割合
concat_data["diff_retire_age"] = concat_data["retirement_age"] - concat_data["current_age"]  # 退職してからの年数
concat_data["period_use_card"] = (concat_data["expires_years"] - concat_data["acct_open_date_years"])*12 + (concat_data["expires_month"] - concat_data["acct_open_date_month"])  # カード使用期間(月)
concat_data["diff_expirs_change_pin"] = concat_data["expires_years"] - concat_data["year_pin_last_changed"]  # pinコードを最後に変更した日から有効期限までの年数
concat_data["diff_change_pin_acct_open"] = concat_data["year_pin_last_changed"] - concat_data["acct_open_date_years"] # アカウント作成日からpinコードを最後に変更した日までの年数
concat_data["acct_oppen_age"] = (concat_data["acct_open_date_years"] - concat_data["birth_year"])*12 + (concat_data["acct_open_date_month"] - concat_data["birth_month"])  # アカウント作成時の年齢(月)

# コンペ開始時期でも使えるカードか否か
concat_data["current_use_card"] = 0
concat_data.loc[concat_data["expires_years"]>=2024, "current_use_card"] = 1
concat_data.loc[((concat_data["expires_years"]>=2023) & (concat_data["expires_month"]>=8)), "current_use_card"] = 1


In [21]:
train_df = concat_data.iloc[:train.shape[0],:]
train_df = train_df.reset_index(drop=True)
test_df = concat_data.iloc[train.shape[0]:,:]
test_df = test_df.reset_index(drop=True)

In [22]:
numerical_features = ["amount", "cards_issued", "credit_limit", "current_age", "retirement_age", "birth_year", "latitude", "longitude",
                      "per_capita_income_zipcode", "yearly_income_person", "total_debt", "fico_score", "num_credit_cards", "income_rate"]
ori_categorical_features = ["user_id", "card_id", "errors?", "merchant_id", "merchant_city", "merchant_state", "mcc", "use_chip",
                        "card_brand", "card_type", "has_chip", "gender", "city", "state",
                        "birth_month",
                        ]
target = "is_fraud?"

In [23]:
amount_user_mean = concat_data.groupby("user_id")["amount"].mean().to_dict()
train_df["amount_mean"] = train_df["user_id"].map(lambda x: amount_user_mean[x]).values
# test_df["amount_mean"] = test_df["user_id"].map(lambda x: amount_user_mean[x]).values
train_df["diff_amount"] = train_df["amount"] - train_df["amount_mean"]
# test_df["diff_amount"] = test_df["amount"] - test_df["amount_mean"]

In [24]:
train_df["neg_is_franud?"] = pd.Series(np.where(train_df[target]==1.0, 0.0, 1.0))
total_pos = train_df[target].sum()
total_neg = train_df["neg_is_franud?"].sum()

for col in ori_categorical_features:
  print(col)
  if col=="merchant_id":
    continue
  pos = train_df.groupby(col)[target].sum() / total_pos
  neg = train_df.groupby(col)["neg_is_franud?"].sum() / total_neg
  woe = pos/neg
  train_df[f"{col}_woe"] = np.log(train_df[col].map(woe).astype(float))
  test_df[f"{col}_woe"] = np.log(test_df[col].map(woe).astype(float))

user_id
card_id
errors?
merchant_id
merchant_city
merchant_state
mcc
use_chip
card_brand
card_type
has_chip
gender
city
state
birth_month


In [25]:
train_df[target] = train_df[target].astype(float)
# corr_list = [target] + numerical_features + [f"{col}_woe" for col in ori_categorical_features if col not in "merchant_id"]
# train_df[corr_list].corr()

In [26]:
class CFG:
  debug=False
  model="lightgbm"
  n_folds=5
  trn_fold=[0, 1, 2, 3, 4]
  seed=42
  target= target
  boosting_type="gbdt"
  early_stopping_round=50
  verbose=200

In [27]:
#skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
#for fold, (train_index, valid_index) in enumerate(skf.split(train_df, train_df[CFG.target])):
#    train_df.loc[valid_index, "fold"] = fold
#train_df['fold'] = train_df['fold'].astype(int)
#display(train_df.groupby('fold').size())

In [28]:
if CFG.debug:
    display(train_df.groupby('kfold').size())
    train_df = train_df.sample(n=1000, random_state=0).reset_index(drop=True)
    display(train_df.groupby('kfold').size())

In [29]:
features = numerical_features + ori_categorical_features + [f"{col}_woe" for col in ori_categorical_features if col not in "merchant_id"]  + ["amount_mean", "diff_amount"] \
+ ["debt_rate", "limit_rate", "amount_rate", "diff_retire_age", "period_use_card", "diff_expirs_change_pin", "diff_change_pin_acct_open", "acct_oppen_age",
 "merchant_address_distance", "credit_limit_sum", "have_card", "acct_open_date_years", "expires_years"] + \
  ["current_use_card", "merchant_address_city","expires_month", "acct_open_date_month", "is_ONLINE", "income_all_rate", "city_c", "state_c"] + \
  ["oof56_logistic_pred", "oof65_ridge_pred", "oof57_knn3_pred", "oof58_knn5_pred", "oof59_knn8_pred", "oof60_knn12_pred", "oof69_rf_pred", "oof74_et_pred"] + \
  ["v3_pred", "v1_pred", "roberta_pred", "funnel_pred", "electra_pred"]
categorical_features = ori_categorical_features +  ["current_use_card", "merchant_address_city","expires_month", "acct_open_date_month"]
oof_features = ["oof56_logistic_pred", "oof65_ridge_pred", "oof57_knn3_pred", "oof58_knn5_pred", "oof59_knn8_pred", "oof60_knn12_pred", "oof69_rf_pred", "oof74_et_pred"] + ["v3_pred", "v1_pred", "roberta_pred", "funnel_pred", "electra_pred"]

In [30]:
def objective(trial):
    params = {
        "learning_rate":0.05,
        "objective": "binary",
        #"metric": "log_loss",
        'boosting_type': "gbdt", #trial.suggest_categorical("boosting_type", ["dart", "gbdt"]),
        'verbosity': -1,
        'n_jobs': -1,
        'seed': CFG.seed,
        #'device':'cuda',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'n_estimators': 5000,
        'max_depth': trial.suggest_categorical('max_depth', [3, 4, 5, 7, 9, 11, 13]),
    }

    oof_df = pd.DataFrame()
    for fold in range(CFG.n_folds):
      if fold in CFG.trn_fold:
        #print("="*15,f" FOLD : {fold} ","="*15)
        train_folds = train_df[train_df['kfold'] != fold].reset_index(drop=True)
        valid_folds = train_df[train_df['kfold'] == fold].reset_index(drop=True)


        for col in oof_features:
          train_folds[col] = train_folds[col].astype(float)
          valid_folds[col] = valid_folds[col].astype(float)
          test_df[col] = test_df[col].astype(float)

        train_X = train_folds[features]
        valid_X = valid_folds[features]
        train_labels = train_folds[CFG.target]
        valid_labels = valid_folds[CFG.target]

        train_dataset = lgb.Dataset(train_X, train_labels)
        valid_dataset = lgb.Dataset(valid_X, valid_labels, reference=train_dataset)

        model = lgb.train(params,
                          train_set = train_dataset,
                          valid_sets = [train_dataset, valid_dataset],
                          callbacks=[
                          lgb.early_stopping(stopping_rounds=CFG.early_stopping_round, verbose=False)],
                          categorical_feature = categorical_features
                          )

        valid_folds["pred"] = model.predict(valid_X)
        oof_df = pd.concat([oof_df, valid_folds])

    pred = np.where(oof_df["pred"]>0.33, 1, 0)
    scoref1 =  f1_score(oof_df[CFG.target], pred)
    return scoref1

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-09-09 21:24:02,164] A new study created in memory with name: no-name-90a15d36-1bcd-4dce-904c-d1c8f6718702
[I 2023-09-09 21:25:42,477] Trial 0 finished with value: 0.6431299011251278 and parameters: {'lambda_l1': 6.6086929825743725, 'lambda_l2': 0.01025211867174334, 'num_leaves': 102, 'feature_fraction': 0.7949708702262694, 'bagging_fraction': 0.9518762000079128, 'bagging_freq': 5, 'min_child_samples': 86, 'max_depth': 3}. Best is trial 0 with value: 0.6431299011251278.
[I 2023-09-09 21:27:03,232] Trial 1 finished with value: 0.6436578871495684 and parameters: {'lambda_l1': 0.0001712822436326871, 'lambda_l2': 0.5387543385015577, 'num_leaves': 47, 'feature_fraction': 0.5717034653693501, 'bagging_fraction': 0.8656153887007738, 'bagging_freq': 3, 'min_child_samples': 25, 'max_depth': 4}. Best is trial 1 with value: 0.6436578871495684.
[I 2023-09-09 21:28:22,342] Trial 2 finished with value: 0.644169899938738 and parameters: {'lambda_l1': 0.2366019456114819, 'lambda_l2': 0.000171782

Number of finished trials: 200
Best trial: {'lambda_l1': 0.025192968836399997, 'lambda_l2': 3.349679988465967e-08, 'num_leaves': 206, 'feature_fraction': 0.8800582352785101, 'bagging_fraction': 0.896935285190445, 'bagging_freq': 3, 'min_child_samples': 86, 'max_depth': 13}
